Drill Hole Compound Likelihood Demo
===================================

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as pl
import numpy as np
import logging

from revrand import glm
from revrand.basis_functions import BiasBasis, RandomRBF
from revrand.btypes import Parameter, Positive
from revrand.utils.datasets import gen_gausprocess_se
from revrand.mathfun.special import softplus

from uncoverml.likelihoods import Switching

# Log output to the terminal attached to this notebook
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

# Warning handling
#np.seterr(all='warn')
import warnings
warnings.filterwarnings('error')


Demo settings
-------------

In [3]:
# Dataset properties
N = 300
Ns = 250
offset = 20
lenscale_true = 0.7  # For the gpdraw dataset
noise_true = 0.1
hit_proportion = 0.0

fig_width = 15
fig_height = 10


Algorithm Settings
------------------

In [4]:
nbases = 100
lenscale = 1  # For all basis functions that take lengthscales
rho = 0.9
epsilon = 1e-5
maxiter = 4000
batch_size = 10
kappa = 6
use_sgd = True
regulariser = 1.
noise = 0.5


Make data
---------

In [5]:
hit = np.random.binomial(n=1, p=hit_proportion, size=N).astype(bool)
not_hit = ~ hit
Xtrain, ftrain, Xtest, ftest = \
    gen_gausprocess_se(N, Ns, lenscale=lenscale_true, noise=0.0)

gtrain = ftrain + offset
gtest = ftest + offset

ytrain = np.empty(N)
ytrain[hit] = gtrain[hit] + np.random.randn(hit.sum()) * noise_true
ytrain[not_hit] = np.random.rand(not_hit.sum()) * gtrain[not_hit]


Set up likelihood and Basis
---------------------------

In [6]:
# Make parameter types
var = Parameter(noise**2, Positive())
regulariser = Parameter(regulariser, Positive())
lenscale_init = Parameter(lenscale, Positive())

# Likelihood
like = Switching(var_init=var)

# Regression Basis
basis = BiasBasis(offset=offset) \
    + RandomRBF(nbases, Xtrain.shape[1], lenscale_init)


Learning and prediction
-----------------------

In [7]:
# Learning
params = glm.learn(Xtrain, ytrain, like, basis, regulariser=regulariser,
                   likelihood_args=(hit,), use_sgd=use_sgd, rho=rho,
                   epsilon=epsilon, batch_size=batch_size, maxiter=maxiter)

# Expected drill hole measurements (proxy for basement depth)
hit_predict = np.ones(Ns, dtype=bool)
Ey, Vy, Eyn, Eyx = glm.predict_moments(Xtest, like, basis, *params,
                                       likelihood_args=(hit_predict,))

# 95% confidence interval
y95n, y95x = glm.predict_interval(0.8, Xtest, like, basis, *params,
                                  multiproc=False,
                                  likelihood_args=(hit_predict,))


RuntimeWarning: overflow encountered in square

In [ ]:
%debug


Visualise
---------

In [ ]:
Xpl_t = Xtrain.flatten()
Xpl_s = Xtest.flatten()

# Regressor
pl.figure(figsize=(fig_width, fig_height))
pl.plot(Xpl_s, Ey, 'b-', label='GLM mean.')
pl.plot(Xpl_s, gmean, 'r-', label='$\mathbf{g}(\mathbf{f})$ mean')
pl.fill_between(Xpl_s, y95n, y95x, facecolor='none', edgecolor='b', label=None,
                linestyle='--')

# # Training/Truth
pl.plot(Xpl_t[hit], ytrain[hit], 'k.', label='Training (hit)')
pl.plot(Xpl_t[not_hit], ytrain[not_hit], 'kx', label='Training (not hit)')
pl.plot(Xpl_s, gtest, 'k-', label='Truth')

pl.gca().invert_yaxis()
pl.legend()
pl.grid(True)
pl.title('Regression demo')
pl.ylabel('depth')
pl.xlabel('x')

pl.show()
